In [165]:
# Load libraries
import gurobipy as gp
from gurobipy import GRB,tuplelist

In [166]:
# Set parameters
P = 0
p = -5    # Defining the accuracy
set_l = range(p,P+1)
set_k = range(10)
set_kl = tuplelist([(k,l) for l in set_l for k in set_k])

m = gp.Model('MDT')

# Define the continuous variables
x1 = m.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x1")
x2 = m.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x2")

# print([i for i in set_kl])

In [167]:
m.update()

w12 = m.addVar(vtype=GRB.CONTINUOUS, name="w12")
Delta_w12 = m.addVar(lb=0, ub=1.5 * x1.UB, vtype=GRB.CONTINUOUS, name="Delta_w12")  # 1.5 * 1.5 upper bound
Delta_x1 = m.addVar(lb=0, ub=1.5 * 10**p, vtype=GRB.CONTINUOUS, name="Delta_x1")

In [168]:
# Define the indexed continuous variables (hat_x_k) and binary variables (z_k)
hat_x = m.addVars(set_kl, lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="hat_x")
z = m.addVars(set_kl, vtype=GRB.BINARY, name="z")

# Set the objective function: Minimize f = -x1 + w12 - x2
m.setObjective(-x1 + w12 - x2, GRB.MINIMIZE)
m.update()
# hat_x

In [169]:
# Constraint 1: -6*x1 + 8*x2 <= 3
m.addConstr(-6*x1 + 8*x2 <= 3, "c1")

# Constraint 2: 3*x1 - x2 <= 3
m.addConstr(3*x1 - x2 <= 3, "c2")

# Constraint 3: w12 = sum(k * hat_x[k]) + Delta_w12
m.addConstr(w12 == gp.quicksum(gp.quicksum(k * hat_x[k,l] for k in set_k) for l in set_l) + Delta_w12, "c3")

# Constraint 4: x1 = sum(10^0 * k * z_k) + Delta_x1
m.addConstr(x1 == gp.quicksum(gp.quicksum(10**p * k * z[k,l] for k in set_k) for l in set_l) + Delta_x1, "c4")

# Constraint 5: x2 = sum(hat_x[k])
m.addConstrs((x2 == gp.quicksum(hat_x[k,l] for k in set_k) for l in set_l), "c5")

# Constraint 6: hat_x[k] <= 1.5 * z[k] for all k
# for k in range(10):
m.addConstrs((hat_x[k,l] <= 1.5 * z[k,l] for k,l in set_kl), "c6")

# Constraint 7: Sum(z_k) = 1
m.addConstrs((gp.quicksum(z[k,l] for k in set_k) == 1 for l in set_l), "c7")

# Constraint 8: 0 <= Delta_w12 <= 1.5 * Delta_x1
m.addConstr(Delta_w12 >= 0, "c8a")
m.addConstr(Delta_w12 <= 1.5 * Delta_x1, "c8b")

# Constraint 9: (x2 - 1.5) * 10^0 + 1.5 * Delta_x1 <= Delta_w12 <= x2 * 10^0
m.addConstr((x2 - 1.5) * 10**p + 1.5 * Delta_x1 <= Delta_w12, "c9a")
m.addConstr(Delta_w12 <= x2 * 10**p, "c9b")

# Constraint 10: 0 <= Delta_x1 <= 1.5 * 10^0
m.addConstr(Delta_x1 >= 0, "c10a")
m.addConstr(Delta_x1 <= 1.5 * 10**p, "c10b")

# Constraint 11: 0 <= hat_x[k] <= 1.5 for all k
# for k in range(10):
#     m.addConstr(hat_x[k] >= 0, f"c11a_{k}")
#     m.addConstr(hat_x[k] <= 1.5, f"c11b_{k}")

# Optimize the m
m.Params.OutputFlag = 1
m.write('kk.lp')
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 82 rows, 125 columns and 372 nonzeros
Model fingerprint: 0x400476b0
Variable types: 65 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e-05, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-05, 2e+00]
  RHS range        [2e-05, 3e+00]
Found heuristic solution: objective -0.0002900
Presolve removed 13 rows and 8 columns
Presolve time: 0.00s
Presolved: 69 rows, 117 columns, 295 nonzeros
Found heuristic solution: objective -0.0005475
Variable types: 63 continuous, 54 integer (54 binary)

Root relaxation: objective -3.750148e-01, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [170]:
m.NumConstrs

82